In [3]:
!pip install catboost
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, r2_score
from sklearn.cluster import KMeans
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.preprocessing import OneHotEncoder

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 9.8 MB/s eta 0:00:00


In [4]:
df = pd.read_csv("/content/drive/MyDrive/ML_kurs2022/data/train_dataset_train.csv",nrows = 100000)
df

,id,ticket_id,ticket_type_nm,entrance_id,entrance_nm,station_id,station_nm,line_id,line_nm,pass_dttm,time_to_under,label
0,1,40BD89EC85646EFB69E283F39C298E60,Пропуск FacePay,2402,Лефортово БКЛ,11007,Лефортово,11,Большая кольцевая,2022-09-12 05:00:13,216.316667,8001
1,2,126727A96489CC976A8C08E5CEB00542,СК учащегося 30 дней,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,2022-09-12 05:00:54,648.183333,9011
2,3,D28CE6A9E0E5B6D213470A97CFF32485,БСК дружинника г.Москвы,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,2022-09-12 05:00:55,865.333333,7022
3,4,015DA44B523C062B5BFEFF3FB0E64B9E,30 дней,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,2022-09-12 05:01:13,1048.233333,2022
4,5,95B19C6F3A504727AC3EA56EB7E3E80F,КОШЕЛЕК,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,2022-09-12 05:02:55,965.600000,2017
...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99996,7CAE7AE97A4A86C6CAD7658BECD417A3,Социальная карта москвича,386,Нагорная,9010,Нагорная,9,Серпуховско-Тимирязевская,2022-09-12 09:01:31,670.483333,1015
99996,99997,A3E49F5C9B42FBA60C5333CBFF3CE04E,СК учащегося 30 дней,300,Рязанский проспект (Восточный),7016,Рязанский проспект,7,Таганско-Краснопресненская,2022-09-12 09:01:31,537.316667,7015
99997,99998,6A316D059D8E67C83165DAD07D6D4B18,30 дней,109,Войковская ( Северный ),2006,Войковская,2,Замоскворецкая,2022-09-12 09:01:31,707.716667,6008
99998,99999,116262B31A4C71884EAC142CEA331E3B,90 дней ЕДИНЫЙ ТК,276,Новые Черёмушки (Южный),6013,Новые Черёмушки,6,Калужско-Рижская,2022-09-12 09:01:31,566.066667,8008


In [5]:
df["station_nm"].unique()

array(['Лефортово', 'Войковская', 'Аэропорт', 'Косино', 'Котельники',
       'Сокол', 'Новогиреево', 'Кунцевская', 'Проспект Вернадского',
       'Динамо', 'Черкизовская', 'Отрадное', 'Крылатское', 'Мнёвники',
       'Хорошёвская', 'Селигерская', 'Павелецкая', 'Первомайская',
       'Орехово', 'Тургеневская', 'Выставочная', 'Кропоткинская',
       'Площадь Ильича', 'Мякинино', 'Лухмановская', 'Свиблово',
       'Бутырская', 'Петровско-Разумовская', 'Владыкино', 'Комсомольская',
       'Таганская', 'Проспект Мира', 'Шипиловская', 'Третьяковская',
       'Выхино', 'Каширская', 'Савёловская', 'Семёновская',
       'Рязанский проспект', 'Чертановская', 'Печатники', 'Балтийская',
       'Преображенская площадь', 'Севастопольская', 'Тимирязевская',
       'Перово', 'Панфиловская', 'Борисово', 'ВДНХ', 'Новые Черёмушки',
       'Партизанская', 'Ботанический сад', 'Лубянка', 'Спартак',
       'Солнцево', 'Аннино', 'Нижегородская', 'Бульвар Рокоссовского',
       'Смоленская', 'Новокосино', 'Лок

In [6]:
date = df["pass_dttm"]

def date2num(str, isDay):
  days = []
  hours = []
  for i in str:
    day = i.split(" ")[0]
    hour = i.split(" ")[1]
    day = day.split("-")[2]
    hour = hour.split(":")[0]
    days.append(int(day))
    hours.append(int(hour))
  if isDay:
    return days
  else:
    return hours
    
df["days"] = date2num(date, 1)
df["hours"] = date2num(date, 0)
df = df.drop(["pass_dttm","id","entrance_nm","entrance_id","ticket_id","station_id","line_id"],axis = 1)
df

,ticket_type_nm,station_nm,line_nm,time_to_under,label,days,hours
0,Пропуск FacePay,Лефортово,Большая кольцевая,216.316667,8001,12,5
1,СК учащегося 30 дней,Войковская,Замоскворецкая,648.183333,9011,12,5
2,БСК дружинника г.Москвы,Войковская,Замоскворецкая,865.333333,7022,12,5
3,30 дней,Войковская,Замоскворецкая,1048.233333,2022,12,5
4,КОШЕЛЕК,Войковская,Замоскворецкая,965.600000,2017,12,5
...,...,...,...,...,...,...,...
99995,Социальная карта москвича,Нагорная,Серпуховско-Тимирязевская,670.483333,1015,12,9
99996,СК учащегося 30 дней,Рязанский проспект,Таганско-Краснопресненская,537.316667,7015,12,9
99997,30 дней,Войковская,Замоскворецкая,707.716667,6008,12,9
99998,90 дней ЕДИНЫЙ ТК,Новые Черёмушки,Калужско-Рижская,566.066667,8008,12,9


In [7]:
from sklearn.feature_extraction import FeatureHasher

In [8]:
h_ticket_type = FeatureHasher(n_features=5, input_type='string')
hashed_Feature = h_ticket_type.fit_transform(df['ticket_type_nm'])
hashed_Feature = hashed_Feature.toarray()
df = pd.concat([df, pd.DataFrame(hashed_Feature)], axis=1)

h_station = FeatureHasher(n_features=5, input_type='string')
hashed_Feature = h_station.fit_transform(df['station_nm'])
hashed_Feature = hashed_Feature.toarray()
df = pd.concat([df, pd.DataFrame(hashed_Feature)], axis=1)

h_line = FeatureHasher(n_features=10, input_type='string')
hashed_Feature = h_line.fit_transform(df['line_nm'])
hashed_Feature = hashed_Feature.toarray()
df = pd.concat([df, pd.DataFrame(hashed_Feature)], axis=1)

df = df.drop(["ticket_type_nm","station_nm","line_nm"],axis=1)
df

,time_to_under,label,days,hours,0,1,2,3,4,0,...,0,1,2,3,4,5,6,7,8,9
0,216.316667,8001,12,5,1.0,-2.0,-1.0,-3.0,2.0,2.0,...,2.0,0.0,1.0,1.0,-3.0,0.0,0.0,-2.0,1.0,1.0
1,648.183333,9011,12,5,0.0,-2.0,2.0,3.0,1.0,-1.0,...,1.0,0.0,0.0,1.0,-2.0,0.0,1.0,-1.0,-1.0,1.0
2,865.333333,7022,12,5,-1.0,-5.0,2.0,1.0,0.0,-1.0,...,1.0,0.0,0.0,1.0,-2.0,0.0,1.0,-1.0,-1.0,1.0
3,1048.233333,2022,12,5,0.0,-1.0,1.0,2.0,-1.0,-1.0,...,1.0,0.0,0.0,1.0,-2.0,0.0,1.0,-1.0,-1.0,1.0
4,965.600000,2017,12,5,-2.0,-1.0,0.0,2.0,2.0,-1.0,...,1.0,0.0,0.0,1.0,-2.0,0.0,1.0,-1.0,-1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,670.483333,1015,12,9,1.0,-3.0,1.0,-1.0,-3.0,1.0,...,0.0,-4.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0
99996,537.316667,7015,12,9,0.0,-2.0,2.0,3.0,1.0,-4.0,...,-2.0,-2.0,0.0,0.0,-4.0,-1.0,-1.0,-1.0,5.0,0.0
99997,707.716667,6008,12,9,0.0,-1.0,1.0,2.0,-1.0,-1.0,...,1.0,0.0,0.0,1.0,-2.0,0.0,1.0,-1.0,-1.0,1.0
99998,566.066667,8008,12,9,-1.0,0.0,1.0,4.0,-1.0,0.0,...,-1.0,-1.0,0.0,-1.0,-1.0,0.0,0.0,-1.0,-1.0,2.0


In [9]:
columns = ["time_to_under","label","days","hours"]
data = df.values

for i in range(len(df.columns)):
  if(type(df.columns[i]) == type(5)):
    columns.append("x" + str(i-3))
df = pd.DataFrame(data, columns = columns)
df

,time_to_under,label,days,hours,x1,x2,x3,x4,x5,x6,...,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20
0,216.316667,8001.0,12.0,5.0,1.0,-2.0,-1.0,-3.0,2.0,2.0,...,2.0,0.0,1.0,1.0,-3.0,0.0,0.0,-2.0,1.0,1.0
1,648.183333,9011.0,12.0,5.0,0.0,-2.0,2.0,3.0,1.0,-1.0,...,1.0,0.0,0.0,1.0,-2.0,0.0,1.0,-1.0,-1.0,1.0
2,865.333333,7022.0,12.0,5.0,-1.0,-5.0,2.0,1.0,0.0,-1.0,...,1.0,0.0,0.0,1.0,-2.0,0.0,1.0,-1.0,-1.0,1.0
3,1048.233333,2022.0,12.0,5.0,0.0,-1.0,1.0,2.0,-1.0,-1.0,...,1.0,0.0,0.0,1.0,-2.0,0.0,1.0,-1.0,-1.0,1.0
4,965.600000,2017.0,12.0,5.0,-2.0,-1.0,0.0,2.0,2.0,-1.0,...,1.0,0.0,0.0,1.0,-2.0,0.0,1.0,-1.0,-1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,670.483333,1015.0,12.0,9.0,1.0,-3.0,1.0,-1.0,-3.0,1.0,...,0.0,-4.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0
99996,537.316667,7015.0,12.0,9.0,0.0,-2.0,2.0,3.0,1.0,-4.0,...,-2.0,-2.0,0.0,0.0,-4.0,-1.0,-1.0,-1.0,5.0,0.0
99997,707.716667,6008.0,12.0,9.0,0.0,-1.0,1.0,2.0,-1.0,-1.0,...,1.0,0.0,0.0,1.0,-2.0,0.0,1.0,-1.0,-1.0,1.0
99998,566.066667,8008.0,12.0,9.0,-1.0,0.0,1.0,4.0,-1.0,0.0,...,-1.0,-1.0,0.0,-1.0,-1.0,0.0,0.0,-1.0,-1.0,2.0


In [10]:
df2 = df.drop(["time_to_under","label","days"],axis=1)
kmeans = KMeans(n_clusters=3, random_state=42).fit(df2)
cluster = kmeans.predict(df2)
df2['cluster'] = cluster

In [11]:
df2

,hours,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x12,x13,x14,x15,x16,x17,x18,x19,x20,cluster
0,5.0,1.0,-2.0,-1.0,-3.0,2.0,2.0,-2.0,0.0,1.0,...,0.0,1.0,1.0,-3.0,0.0,0.0,-2.0,1.0,1.0,0
1,5.0,0.0,-2.0,2.0,3.0,1.0,-1.0,0.0,-1.0,-1.0,...,0.0,0.0,1.0,-2.0,0.0,1.0,-1.0,-1.0,1.0,1
2,5.0,-1.0,-5.0,2.0,1.0,0.0,-1.0,0.0,-1.0,-1.0,...,0.0,0.0,1.0,-2.0,0.0,1.0,-1.0,-1.0,1.0,0
3,5.0,0.0,-1.0,1.0,2.0,-1.0,-1.0,0.0,-1.0,-1.0,...,0.0,0.0,1.0,-2.0,0.0,1.0,-1.0,-1.0,1.0,1
4,5.0,-2.0,-1.0,0.0,2.0,2.0,-1.0,0.0,-1.0,-1.0,...,0.0,0.0,1.0,-2.0,0.0,1.0,-1.0,-1.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,9.0,1.0,-3.0,1.0,-1.0,-3.0,1.0,0.0,-1.0,2.0,...,-4.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,0
99996,9.0,0.0,-2.0,2.0,3.0,1.0,-4.0,-2.0,-1.0,0.0,...,-2.0,0.0,0.0,-4.0,-1.0,-1.0,-1.0,5.0,0.0,2
99997,9.0,0.0,-1.0,1.0,2.0,-1.0,-1.0,0.0,-1.0,-1.0,...,0.0,0.0,1.0,-2.0,0.0,1.0,-1.0,-1.0,1.0,1
99998,9.0,-1.0,0.0,1.0,4.0,-1.0,0.0,0.0,2.0,2.0,...,-1.0,0.0,-1.0,-1.0,0.0,0.0,-1.0,-1.0,2.0,1


In [ ]:
xl = df['label']
yt = df["time_to_under"]
clf = CatBoostClassifier(iterations=25)
clf.fit(df2, xl)

Learning rate set to 0.5
0:	learn: 5.3055255	total: 12.9s	remaining: 5m 10s
1:	learn: 5.1566592	total: 25.2s	remaining: 4m 49s
2:	learn: 5.0934195	total: 36.9s	remaining: 4m 30s
3:	learn: 5.0216717	total: 49.2s	remaining: 4m 18s
4:	learn: 4.9591183	total: 1m 1s	remaining: 4m 6s
5:	learn: 4.9257557	total: 1m 13s	remaining: 3m 52s
6:	learn: 4.8686049	total: 1m 25s	remaining: 3m 39s
7:	learn: 4.8296023	total: 1m 38s	remaining: 3m 29s
8:	learn: 4.8045381	total: 1m 51s	remaining: 3m 17s
9:	learn: 4.7767046	total: 2m 3s	remaining: 3m 4s
10:	learn: 4.7449220	total: 2m 14s	remaining: 2m 51s
11:	learn: 4.7303551	total: 2m 26s	remaining: 2m 38s
12:	learn: 4.7057626	total: 2m 38s	remaining: 2m 26s
13:	learn: 4.6822446	total: 2m 50s	remaining: 2m 13s
14:	learn: 4.6726628	total: 3m 1s	remaining: 2m 1s
15:	learn: 4.6626813	total: 3m 13s	remaining: 1m 48s
16:	learn: 4.6514505	total: 3m 25s	remaining: 1m 36s
17:	learn: 4.6442722	total: 3m 36s	remaining: 1m 24s
18:	learn: 4.6339629	total: 3m 48s	remain

In [ ]:
pred = clf.predict(df2)
reg = CatBoostRegressor(iterations=1000)
reg.fit(df2, yt)

Learning rate set to 0.084758
0:	learn: 157.0287067	total: 26.1ms	remaining: 26.1s
1:	learn: 155.1674011	total: 51.4ms	remaining: 25.6s
2:	learn: 153.5746793	total: 71.3ms	remaining: 23.7s
3:	learn: 152.2179538	total: 94.2ms	remaining: 23.5s
4:	learn: 151.1013672	total: 126ms	remaining: 25s
5:	learn: 150.1547049	total: 161ms	remaining: 26.6s
6:	learn: 149.3505159	total: 200ms	remaining: 28.4s
7:	learn: 148.6357218	total: 235ms	remaining: 29.2s
8:	learn: 148.0446378	total: 268ms	remaining: 29.5s
9:	learn: 147.5328012	total: 320ms	remaining: 31.6s
10:	learn: 147.1060084	total: 376ms	remaining: 33.8s
11:	learn: 146.7682437	total: 433ms	remaining: 35.6s
12:	learn: 146.4527334	total: 472ms	remaining: 35.8s
13:	learn: 146.1849299	total: 494ms	remaining: 34.8s
14:	learn: 145.9573580	total: 531ms	remaining: 34.9s
15:	learn: 145.7501992	total: 582ms	remaining: 35.8s
16:	learn: 145.5600542	total: 644ms	remaining: 37.3s
17:	learn: 145.3878871	total: 699ms	remaining: 38.2s
18:	learn: 145.2669756	t

In [ ]:
pred = clf.predict(df2)
pred_reg = reg.predict(df2)

r2 = r2_score(yt, pred_reg)
r2


0.2236042899324725

In [ ]:
recall = recall_score(xl, pred, average="micro")
recall

0.08212

In [ ]:
score = 0.5*(recall+r2)
score

0.15286214496623624